In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bluebook-for-bulldozers/TrainAndValid.csv
/kaggle/input/bluebook-for-bulldozers/Data Dictionary.xlsx
/kaggle/input/bluebook-for-bulldozers/random_forest_benchmark_test.csv
/kaggle/input/bluebook-for-bulldozers/ValidSolution.csv
/kaggle/input/bluebook-for-bulldozers/Test.csv
/kaggle/input/bluebook-for-bulldozers/Machine_Appendix.csv
/kaggle/input/bluebook-for-bulldozers/Valid.csv
/kaggle/input/bluebook-for-bulldozers/median_benchmark.csv
/kaggle/input/bluebook-for-bulldozers/valid/Valid.csv
/kaggle/input/bluebook-for-bulldozers/trainandvalid/TrainAndValid.csv
/kaggle/input/bluebook-for-bulldozers/train/Train.csv


In [2]:
from fastai.imports import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import IPython as Ipython
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn import metrics
from pathlib2 import Path
import feather
from sklearn.tree import export_graphviz
import re
import graphviz

In [3]:
path = Path('/kaggle/input/bluebook-for-bulldozers')
list(path.iterdir())

[PosixPath('/kaggle/input/bluebook-for-bulldozers/TrainAndValid.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/valid'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/trainandvalid'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/Data Dictionary.xlsx'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/random_forest_benchmark_test.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/ValidSolution.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/Test.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/Machine_Appendix.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/Valid.csv'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/train'),
 PosixPath('/kaggle/input/bluebook-for-bulldozers/median_benchmark.csv')]

In [4]:
df_raw = pd.read_csv(f'{path}/TrainAndValid.csv', low_memory=False, parse_dates=['saledate'])

In [5]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
        display(df)

In [6]:
print(df_raw.shape)
display_all(df_raw.tail().T)

(412698, 53)


,412693,412694,412695,412696,412697
SalesID,6333344,6333345,6333347,6333348,6333349
SalePrice,10000,10500,12500,10000,13000
MachineID,1919201,1882122,1944213,1794518,1944743
ModelID,21435,21436,21435,21435,21436
datasource,149,149,149,149,149
auctioneerID,2,2,2,2,2
YearMade,2005,2005,2005,2006,2006
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2012-03-07 00:00:00,2012-01-28 00:00:00,2012-01-28 00:00:00,2012-03-07 00:00:00,2012-01-28 00:00:00


In [7]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,412698,NaN,NaN,NaN,NaT,NaT,2.01116e+06,1.08007e+06,1.13925e+06,1.4219e+06,1.64585e+06,2.26101e+06,6.33335e+06
SalePrice,412698,NaN,NaN,NaN,NaT,NaT,31215.2,23141.7,4750,14500,24000,40000,142000
MachineID,412698,NaN,NaN,NaN,NaT,NaT,1.23006e+06,453953,0,1.08859e+06,1.2844e+06,1.47808e+06,2.48633e+06
ModelID,412698,NaN,NaN,NaN,NaT,NaT,6947.2,6280.82,28,3261,4605,8899,37198
datasource,412698,NaN,NaN,NaN,NaT,NaT,135.169,9.64675,121,132,132,136,173
auctioneerID,392562,NaN,NaN,NaN,NaT,NaT,6.58527,17.1584,0,1,2,4,99
YearMade,412698,NaN,NaN,NaN,NaT,NaT,1899.05,292.19,1000,1985,1995,2001,2014
MachineHoursCurrentMeter,147504,NaN,NaN,NaN,NaT,NaT,3522.99,27169.9,0,0,0,3209,2.4833e+06
UsageBand,73670,3,Medium,35832,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,412698,4013,2009-02-16 00:00:00,1932,1989-01-17,2012-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

### Initial processing

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice)

In [ ]:
def parallel_trees(m, fn, n_jobs=8):
    return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    s = export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                        special_characters=True, rotate=True, precision=precision)
    Ipython.display.display(re.sub(graphviz.Source('Tree {', f'Tree {{ size = {size}; ratio={ratio}', s)))
    
def rf_feature_imp(m, df):
    return pd.DataFrame({'cols': df.columns, 'importance': m.feature_importances_}).sort_values('importance', ascending=False)

def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
        
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time:
        attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr:
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop:
        df.drop(fldname, axis=1, inplace=True)
    
def train_cats(df):
    for n, c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype('category').cat.as_ordered()

        
def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = col.cat.codes+1
        
        
def get_sample(df, n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()


def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def proc_df(df, y_fld, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    if not skip_flds:
        skip_flds = []
    if not ignore_flds:
        ignore_flds = []
    if subset:
        df = get_sample(df, subset)
    else:
        df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn:
        preproc_fn(df)
    if y_fld is None:
        y = None
    else:
        if not is_numeric_dtype(df[y_fld]):
            df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)
    
    if na_dict is None:
        na_dict = {}
    else:
        na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n, c in df.items():
        na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale:
        mapper = scale_vars(df, mapper)
    for n, c in df.items():
        numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale:
        res = res + mapper
    return res

def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {', f'Tree {{ size = {size}; ratio={ratio}', s)))
    
def parallel_trees(m, fn, n_jobs=8):
    return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

def rf_feature_imp(m, df):
    return pd.DataFrame({'cols': df.columns, 'importance': m.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
add_datepart(df_raw, 'saledate')

In [ ]:
df_raw.saleYear.head()

In [ ]:
train_cats(df_raw)


In [ ]:
df_raw.UsageBand.cat.categories

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [ ]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [ ]:
display_all(df_raw.isnull().sum().sort_index() / len(df_raw))

In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [ ]:
!ls ../

### Pre-processing

In [ ]:
# df_raw = feather.read_dataframe('tmp/bulldozer-raw')

In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
rmse = lambda x, y: math.sqrt(((x - y) ** 2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train),
           m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'):
        res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

In [ ]:
def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()

In [ ]:
n_valid = 12000
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
df_raw.head()

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

In [ ]:
draw_tree??